In [1]:
import os
import json
import requests
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key:
    print('API key loaded')
else:
    print('API key loading issue')

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key loaded


In [3]:
class Website:
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ""
        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage title:\n{self.title}\nWebpage contents:\n{self.text}\n"

In [4]:
links_sys_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which one to to pick from the list to be most relevent to include in a brouchure about the company,
such as links to an About page or a company page or careers/jobs page.\n

You should response in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url.link/about"},
        {"type": "careers page", "url": "https://another.ful.url/careers"}
    ]
}
"""


In [5]:
def get_links_user_prompt(website):
    user_prompt = f"""
    Here is the list of links on the website of {website.url} - 
    please decide which of these are relevent web links for a brochure about the company, response with the full https url \n
    Do not include Terms of Service, Privacy, email links\n
    Here are the links: {website.links}
    """

    return user_prompt

In [6]:
def get_links(url):
    w = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": links_sys_prompt},
            {"role": "user", "content": get_links_user_prompt(w)}
        ],
        response_format={"type":"json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links: ", links)
    for link in links["links"]:
        result += f"\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [8]:
sys_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website and create a short \
brochure about the company for prospective customers, investors and recruits. Respond in Markdown.
Include details of company culture, customers and careers/jobs if you have information.
"""

In [9]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
    You are looking at a company called: {company_name}\n
    Here are the contents of its landing page and other relevent pages; use this information to build a short \
    brochure of the company.
    {get_all_details(url)}
    """
    user_prompt = user_prompt[:20_000]
    return user_prompt

In [10]:
def create_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    # result = response.choices[0].message.content
    # display(Markdown(result))

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [11]:
create_brochure("Google Gemini","https://gemini.google/overview/gemini-in-chrome/")

Links:  {'links': [{'type': 'about page', 'url': 'https://gemini.google/about/'}, {'type': 'overview page', 'url': 'https://gemini.google/overview/'}, {'type': 'our approach page', 'url': 'https://gemini.google/our-approach/'}, {'type': 'latest news page', 'url': 'https://gemini.google/latest-news/'}, {'type': 'careers page', 'url': 'https://www.google.com'}]}


# Welcome to Google Gemini

### Your Everyday AI Assistant

Google Gemini is an innovative AI assistant designed to enhance your browsing experience and streamline your daily tasks. Whether you're a student, a professional, or a curious thinker, Gemini is here to help you make informed choices, learn complex topics, and effortlessly engage with content.

---

### What Can Gemini Do?

#### **AI Assistance in Chrome**
- **Concise Summaries**: Get instant key takeaways from articles, threads, or pages.
- **Curiosity Driven**: Ask questions directly relevant to your open tabs for tailored responses.
- **Deep Research**: Dive into extensive topics and gain a comprehensive understanding without switching tabs.

#### **Gemini Live**
- **Interactive Brainstorming**: Talk through ideas and receive spoken responses.
- **Multi-App Integration**: Utilize Gemini across Google apps like Gmail, Calendar, and Maps.

#### **Creative Tools**
- **Image & Video Generation**: Create stunning visuals and video content with just a prompt using our advanced models.
- **Personalized Learning**: Develop custom study plans, quizzes, and more with your personal AI collaborator.

#### **Custom Experts with Gems**
- Build your unique AI experts tailored to help you tackle specific tasks or areas of interest, whether it’s career coaching, brainstorming, or coding help.

---

### Who Is Our Customer?

Gemini caters to a diverse audience:
- **Students** looking for study assistance and insightful learning tools.
- **Professionals** seeking to enhance productivity and engagement with data.
- **Creative Individuals** who need a quick way to generate ideas and concepts.
- **Curious Minds** eager to explore new topics with clarity.

---

### Company Culture

At Google Gemini, we thrive in an atmosphere of creativity and innovation. Our mission is to empower our users by putting AI assistance at their fingertips—intuitive, smart, and respectful of user control and privacy. We believe in fostering an inclusive workplace that values diverse perspectives, encourages collaboration, and supports continuous learning and growth.

---

### Join the Gemini Team

We are always seeking passionate individuals who share our vision of revolutionizing the way people interact with technology. Here at Google Gemini, you will have the opportunity to innovate, collaborate, and grow in an environment that supports your professional development and respects your work-life balance.

**Explore Careers**: Join us in making AI accessible and empowering for everyone.

---

### Connect with Us
Stay updated with the latest features, enhancements, and community insights:
- **Social Media**: Follow us on Instagram, YouTube, X (Twitter), and Discord!
- **Support**: We offer dedicated support and community engagement to enhance your experience with Gemini.

---

**Discover Gemini Today!**

Ready to transform your online experience? Head over to [gemini.google.com](https://gemini.google.com) to start exploring the future of AI-assisted browsing. Try it free or explore our subscription options for enhanced features!

---

> **Gemini: Intelligence that works with you, right where you are!**